In [208]:

import pandas as pd
import os
from openpyxl import load_workbook
import numpy as np
import re
import datetime as dt

In [209]:

#tisztítás (dátumok)
#dd/mm/yyyy a kívánt formátum
##II->11
df=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\df\\dfveg_latest.xlsx")
otket=[]
for x in df[5]:
    if str(x).startswith("2003"):
        otket.append("03-10-14")

    else:
        otket.append(str(x).replace("I", "1"))


for x in otket:
    if "I" in x:
        print(x)
len(otket)
i=[]
df['otket']=otket
df['otket']=pd.to_datetime(df['otket'], format='mixed', dayfirst=True)

#latlong converter
from lat_lon_parser import parse
df[7].head()
lat=[]
long=[]
df[7]=df[7].fillna("N/A")
df[8]=df[8].fillna("N/A")
for x, y in zip(df[7], df[8]):
    if str(x)!="nan" and str(x)!="N/A" and str(x)!="NaN":
        lat.append(parse(str(x)))
        long.append(parse(str(y)))
    else:
        lat.append(str(x))
        long.append(str(y))

In [210]:
df["lat1"]=lat
df['long']=long

In [211]:



df.sort_values(by=['otket'], inplace=True)
df[6].fillna("N/A")
dfh=[]
for x in df[6]:
    if str(x)=="nan":
        dfh.append("N/A")
    else:
        dfh.append(x)

df[6]=dfh



In [212]:
df['otket'] = pd.to_datetime(df['otket'], errors='coerce')


df['otket'] = df['otket'].dt.strftime('%Y-%m-%d')

df.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\df\\dfveg_latest_2.xlsx")
df['otket'].iloc[980:1000]


980     2016-01-23
981     2016-01-25
982     2016-01-28
983     2016-02-08
984     2016-02-14
985     2016-02-15
986     2016-02-19
987     2016-03-04
988     2016-03-22
989     2016-03-26
990     2016-04-01
991     2016-04-01
992     2016-04-03
993     2016-04-06
994     2016-04-10
995     2016-04-12
996     2016-04-15
997     2016-04-16
1049    2016-04-17
998     2016-04-19
Name: otket, dtype: object

In [204]:
#geodata time
import math
import openmeteo_requests
from retry_requests import retry
import requests_cache

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


cl=[]
t2= []
wss=[]
wsss=[]
wcc=[]
wgg=[]
rr=[]
ind=[]
aaa=0
for x, y, z, k, l in zip(df.lat1, df.long, df.otket, df[6], df.index):
    aaa=aaa+1
    if z!="N/A" and x!="N/A" and y!="N/A" and k!="N/A":
    
        openmeteo = openmeteo_requests.Client()
        url = "https://archive-api.open-meteo.com/v1/archive"
        lista=[]
        params = {
        "latitude": {x},
        "longitude": {y},
        "start_date": {str(z)},
        "end_date": {str(z)},
        "timezone_abbreviation": "SGT",
        "hourly": ["cloud_cover_low", "temperature_2m", "wind_speed_10m", "wind_speed_100m", "weather_code", "wind_gusts_10m", "rain"],
        #"models": "era5"

        }


        res = openmeteo.weather_api(url, params=params)

        hourly=res[0].Hourly()

        ccl=hourly.Variables(0).ValuesAsNumpy()
        temp=hourly.Variables(1).ValuesAsNumpy()
        ws1=hourly.Variables(2).ValuesAsNumpy()
        ws2=hourly.Variables(3).ValuesAsNumpy()
        wc=hourly.Variables(4).ValuesAsNumpy()
        
        wg=hourly.Variables(5).ValuesAsNumpy()
        r=hourly.Variables(6).ValuesAsNumpy()
        i=0
    
        print(aaa)

        for a, b, c, d, e, f, g in zip(ccl, temp, ws1, ws2, wc, wg, r):
            if k!="N/A":
                if i==int(k[:2]):
                    cl.append(a)
                    t2.append(b)
                    wss.append(c)
                    wsss.append(d)
                    wcc.append(e)
                    wgg.append(f)
                    rr.append(g)
                    ind.append(l)
            i+=1
    else:
        a="N/A"    
        cl.append(a)
        t2.append(a)
        wss.append(a)
        wsss.append(a)
        wcc.append(a)
        wgg.append(a)
        rr.append(a)
        ind.append(l)

        
        



print(cl)
print(t2)
print(wss)
print(wsss)
print(wcc)
print(wgg)
print(rr)

1
2
4
5
6
7
9
10
11
12
13
15
16
17
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
118
119
120
121
122
123
124
125
126
127
128
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
169
170
171
172
173
174
176
177
178
179
180
183
184
185
186
187
189
191
192
193
194
195
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
215
216
217
218
219
220
221
222
224
225
226
227
228
229
230
232
234
236
237
238
239
240
241
243
247
248
249
252
253
254
255
256
257
259
260
261
263
264
265
267
268
269
270
271
272
273
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308


In [205]:
for x in df[0]:
    if x not in ind:
        print(x)

df[df[0]==120]

,0,2,3,4,5,6,7,8,9,ship type 2,otket,lat1,long
120,120,121,CAT 3,trust Punner Oil tanker Marshal islands 89136...,21/11/2010,N/A,21° 50' N,091° 39' E,"Kutubdia Anchorage, Bangladesh",&OIL TANKER,2010-11-21,21.833333,91.65


In [207]:
df2=pd.DataFrame()
df2['cloud_cover_low']=cl
df2['temperature_2m']=t2
df2["wind_speed_10m"]=wss
df2["wind_speed_100m"]=wsss
df2["weather_code"]=wcc
df2["wind_gusts_10m"]=wgg
df2["rain"]=rr

df2.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\api_ideig.xlsx")

